In [ ]:
##Imports + functions
import xlwings as xw
import pandas as pd
import numpy as np
import os
import glob
import string
import re
from datetime import date
from datetime import datetime
from tqdm import tqdm

def n2a(n,b=string.ascii_uppercase):
   d, m = divmod(n,len(b))
   return n2a(d-1,b)+b[m] if d else b[m]

def a2n(col):
    num = 0
    for c in col:
        if c in string.ascii_letters:
            num = num * 26 + (ord(c.upper()) - ord('A')) + 1
    return num

def GenerateFilepath(fp):
    fp = fp.replace("\\", "/")
    return fp

def getLatestFile(fp, how="max"):
    if how=="max":
        to_return = max(glob.glob(fp), key=os.path.getctime)
    elif how=="min":
        to_return = min(glob.glob(fp), key=os.path.getctime)
    return to_return

def RowColumnSetup(df, wb):
    df = pd.DataFrame(wb.sheets[sheet].used_range.formula)
    use_range = str(wb.sheets[sheet].used_range).split("!")[1].replace(">","").replace(":","").split("$")[1:]
    df.columns = list(range(a2n(use_range[0])-1, a2n(use_range[2])))
    starting_columns = df.columns
    if starting_columns[0] > 0:
        for i in list(range(0, starting_columns[0])):
            df.insert(i, i, "")
    df.index = list(range(int(use_range[1])-1, int(use_range[3])))
    starting_rows = df.index
    if starting_rows[0] > 0:
        extra_rows = pd.DataFrame("Pikachu", columns=df.columns, index=list(range(0, starting_rows[0])))
        df = pd.concat([extra_rows, df])
        df = df.replace("Pikachu", "")
    return df

# pd.set_option("display.max_rows", None)
# pd.set_option("display.max_columns", None)

In [ ]:
##Initialise
oldfile = GenerateFilepath(r"PATH0") ##Previous version file
newfile = GenerateFilepath(r"PATH1") ##new version file
formulas = True

old_wb = xw.Book(oldfile)
old_sheetslist = [sh.name for sh in old_wb.sheets]
new_wb = xw.Book(newfile)
new_sheetslist = [sh.name for sh in new_wb.sheets]

new_sheets = [x for x in new_sheetslist if x not in old_sheetslist]
old_sheets = [x for x in old_sheetslist if x not in new_sheetslist]
shared_sheets = [x for x in new_sheetslist if x in old_sheetslist]

    ##change log gets written into a column in Start_Nav
change_log = []
if new_sheets:
    change_log.append("Added sheets: "+", ".join(new_sheets))
if old_sheets:
    change_log.append("Removed sheets: "+", ".join(old_sheets))

    ##save version of template spreadsheet to analyse/highlight:
highlightFile = GenerateFilepath(newfile + " HIGHLIGHTED {}.xlsx".format(date.today().strftime("%b-%d-%Y"))) ##keeping original filetype tag in the name to prevent errors/accidentally saving over the original
new_wb.save(highlightFile)


In [ ]:
##Main loop + cleanup:
change_dict = {}
for sheet in shared_sheets:
    # if sheet == "Sheet 1" or sheet == "Sheet 2":
    #     continue ##not excluding these sheets could lead to a lot of keeping highlighted changes that we don't actually need to monitor, plus Link In can be time consuming for the code to highlight

    if formulas:
        ##generating matrix of differences:
        new_df = pd.DataFrame(new_wb.sheets[sheet].used_range.formula)
        old_df = pd.DataFrame(old_wb.sheets[sheet].used_range.formula)
    else:
        new_df = pd.DataFrame(new_wb.sheets[sheet].used_range.value)
        old_df = pd.DataFrame(old_wb.sheets[sheet].used_range.value)

    ##Making sure columns and rows are correct - these functions basically line up the active ranges so rows and columns both start at 0 by adding empty ones if needed
    ##DO ROW SETUP BEFORE COLUMN SETUP! Otherwise empty columns inserted in the ColumnSetup step may disappear
    new_df = RowColumnSetup(new_df, new_wb)
    old_df = RowColumnSetup(old_df, old_wb)

        ##reshaping the dataframes so both have the same number of rows and columns, expanding to the higher of the two on both axes. Before this patch excel would highlight whole sections if one df was larger than the other.
    expanded_cols = old_df.columns.tolist() if old_df.shape[1] > new_df.shape[1] else new_df.columns.tolist()
    expanded_rows = old_df.index.tolist() if old_df.shape[0] > new_df.shape[0] else new_df.index.tolist()
    shape_df = pd.DataFrame(columns=expanded_cols, index=expanded_rows)
    new_df = new_df.reindex_like(shape_df).fillna("")
    old_df = old_df.reindex_like(shape_df).fillna("")

        ##matrix of True/False values - False == formulas are different between both sheets
    match_matrix = new_df.eq(old_df)

        ##getting excel ranges of differences - chunking for better performance in the highlighting stage basically - not currently set up to print the ranges to Start_NAV, but could do that later:
    cols_masterlist = []

    for v in match_matrix.columns:
        col_as_list = match_matrix[v].tolist()
        col_as_list = [[str(match_matrix.columns[v]), str(idx+1), str(x)[0]] for (idx, x) in enumerate(col_as_list)] ##["A", "1", "T"] (actually ["0", "1", "T"], but using Excel notation in comments for demonstration purposes)

        range_list = []
        tfFlag = True
        for i in col_as_list:
            if i[-1] == "F" and tfFlag == True:
                range_list.append([i[0], i[1], i[1], i[0]]) ##["A", 1, 1, "A"]
                tfFlag = False
            elif i[-1] == "F" and tfFlag == False:
                range_list[-1][2] = i[1] ##["A", 1, 2, "A"]
                tfFlag = False
            elif i[-1] == "T":
                tfFlag = True
        cols_masterlist.append(range_list)

    cols_masterlist = [x for lst in cols_masterlist for x in lst] ##flatten list

    final_list = []
    for n in cols_masterlist:
        n = [int(o) for o in n]
        if final_list:
            has_updated = False
            for f in final_list:
                if f[1] == n[1] and f[2]==n[2] and f[3] == n[3]-1:
                    f[3] = n[3] ##["A", 1, 2, "B"]
                    has_updated = True
            if has_updated == False:
                final_list.append(n)
        else:
            final_list.append(n)

    final_list = ["{}{}:{}{}".format(n2a(x[0]), x[1], n2a(x[3]), x[2]) for x in final_list] ##"A1:B2"
    for idx, x in enumerate(final_list): 
        tmpsplit = x.split(":")
        if tmpsplit[0] == tmpsplit[1]:
            final_list[idx] = tmpsplit[0] ##this bit converts e.g. "C3:C3" >> "C3"

        ##updating the change dictionary
    if final_list:
        change_dict.update({sheet: final_list}) ##sheet is sheet name

    ##highlighting edited ranges:
if len(change_dict) > 0:
    change_log.append("Updated sheets:")
    for k, v in change_dict.items():
        change_log.append(k) ##just appending the sheet names with changes for now, could set up to print the actual ranges changes were made in later
        print(k)
        for i in tqdm(v):
            new_wb.sheets[k].range(i).color = (0, 0, 0) ##background fill black
            new_wb.sheets[k].range(i).font.color = (255, 255, 255) ##text colour white

    ##writing change_log to Start_Nav, saving + closing
# new_wb.sheets["Sheet 1"].range("F2").options(transpose=True).value = change_log
# changeflag = " - no changes"
if change_log:
    new_wb.save(highlightFile)
    # changeflag = " - WITH CHANGES"
new_wb.close()
old_wb.close()

In [ ]:
##if you want to check all updated ranges:
for k, v in change_dict.items():
    print("{}: {}".format(k, v))